### Analize and import wifi dataset into Elastic

The primary purpose of this notebook is to do a preliminary data cleaning then import the data into `Elasticsearch.`

In [32]:
%%html
<style>
table {float:left}
</style>

In [33]:
import csv
import json

import numpy as np
import pandas as pd

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

Library | Explanation
-|-
numpy |  open source library enables data analysis in numerical computing 
pandas | open source libraries  based on numpy, used for data structure analysis
elasticseach | pandas library for elasticsearch interaction
elasticsearch.helpers(bulk) | Collection of simple helper functions that abstract some specifics or the raw AP

In [34]:
df=pd.read_csv("wardrive_elasticsearch-01.kismet.csv", delimiter=";")

### Visualize the first 5 records

In [35]:
df.head()

,Network,NetType,ESSID,BSSID,Info,Channel,Cloaked,Encryption,Decrypted,MaxRate,...,GPSMaxLon,GPSMaxAlt,GPSMaxSpd,GPSBestLat,GPSBestLon,GPSBestAlt,DataSize,IPType,IP,Unnamed: 38
0,1,infrastructure,Patricia,C8:D7:19:5D:26:25,NaN,10,No,"WPA2,WPA,AES-CCM,TKIP",No,130.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0.0.0,NaN
1,2,infrastructure,bzn,4C:F9:5D:A6:25:B4,NaN,6,No,"WPA2,AES-CCM",No,130.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0.0.0,NaN
2,3,infrastructure,ASUSR,2C:56:DC:F9:E4:60,NaN,1,No,"WPA2,AES-CCM",No,130.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0.0.0,NaN
3,4,infrastructure,Netis 14 Spalatorie,04:8D:38:7E:C4:37,NaN,9,No,"WPA2,AES-CCM,TKIP",No,270.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0.0.0,NaN
4,5,infrastructure,edimax,80:1F:02:E6:00:1F,NaN,2,No,"WPA2,AES-CCM",No,135.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0.0.0,NaN


### Check dataset dimension(row / column number)

In [36]:
df.shape

(5248, 39)

Shape | explanation
-|-
Rows | 5248
Columns | 39

### Get a list of columns

In [37]:
df.columns

Index(['Network', 'NetType', 'ESSID', 'BSSID', 'Info', 'Channel', 'Cloaked',
       'Encryption', 'Decrypted', 'MaxRate', 'MaxSeenRate', 'Beacon', 'LLC',
       'Data', 'Crypt', 'Weak', 'Total', 'Carrier', 'Encoding', 'FirstTime',
       'LastTime', 'BestQuality', 'BestSignal', 'BestNoise', 'GPSMinLat',
       'GPSMinLon', 'GPSMinAlt', 'GPSMinSpd', 'GPSMaxLat', 'GPSMaxLon',
       'GPSMaxAlt', 'GPSMaxSpd', 'GPSBestLat', 'GPSBestLon', 'GPSBestAlt',
       'DataSize', 'IPType', 'IP', 'Unnamed: 38'],
      dtype='object')

### Filter out columns

Columns such us `GPSMaxLon`, `GPSMaxAlt`, `Decrypted`, `IP` are not useful for our analysis. We create a subset of the original data by specifying a list of columns

In [38]:
df_short=df[['Network', 'NetType', 'ESSID', 'BSSID','Channel','Encryption','MaxRate']]

In [39]:
### Check `NaN` values in the sliced dataset

In [40]:
df_short.isnull().sum()

Network         0
NetType         0
ESSID         173
BSSID           0
Channel         0
Encryption      0
MaxRate         0
dtype: int64

Within pandas,  `NaN,` represent a missing value. `ESSID` column has 173 missing values, and we either have to drop those rows and replace `NaN` with `blank` otherwise we could not import the dataset in elasticsearch. Elasticsearch interprets `NaNs` as `NULL byte`. I chose to drop those records.

#### NaN char:
![IMG](https://github.com/mpruna/Wifi_analysis_using_Elk_Stack/blob/master/images/NaN.png)

#### Import error:
![IMG](https://github.com/mpruna/Wifi_analysis_using_Elk_Stack/blob/master/images/import_error.png)

In [41]:
# Drop ESSID NaNs

df_short=df_short.dropna()

### Check NaN records after drop

In [42]:
df_short.isnull().sum()

Network       0
NetType       0
ESSID         0
BSSID         0
Channel       0
Encryption    0
MaxRate       0
dtype: int64

In [43]:
df_short.head()

,Network,NetType,ESSID,BSSID,Channel,Encryption,MaxRate
0,1,infrastructure,Patricia,C8:D7:19:5D:26:25,10,"WPA2,WPA,AES-CCM,TKIP",130.0
1,2,infrastructure,bzn,4C:F9:5D:A6:25:B4,6,"WPA2,AES-CCM",130.0
2,3,infrastructure,ASUSR,2C:56:DC:F9:E4:60,1,"WPA2,AES-CCM",130.0
3,4,infrastructure,Netis 14 Spalatorie,04:8D:38:7E:C4:37,9,"WPA2,AES-CCM,TKIP",270.0
4,5,infrastructure,edimax,80:1F:02:E6:00:1F,2,"WPA2,AES-CCM",135.0


In [44]:
df_short['ESSID'].value_counts().head(20)

UPC Wi-Free              522
netis                     21
ASUS                      18
dlink                     12
AndroidAP                 11
UVT-Guest                 11
TP-LINK                   10
PMT-Presa                  9
PMT-Guest                  9
Telekom_FON                8
WirelessNet                8
eduroam                    8
PMT                        8
Alex                       7
linksys                    6
GlobalCorpnetWireless      6
NH Timisoara               6
Tenda                      6
BusyMachines               5
Home                       5
Name: ESSID, dtype: int64

In [45]:
df_short['Encryption'].value_counts()

WPA2,AES-CCM             2005
WPA2,AES-CCM,TKIP        1358
WPA2,WPA,AES-CCM          799
WPA2,WPA,AES-CCM,TKIP     557
OPN,None                  203
WPA,AES-CCM                72
WEP                        27
WPA,TKIP                   21
WPA2,TKIP                  20
WPA,AES-CCM,TKIP            8
WPA2,WPA,TKIP               2
WPA2,OPN,AES-CCM,TKIP       2
WPA2,None                   1
Name: Encryption, dtype: int64

In [46]:
### Import dataset into elastic

In [47]:
es = Elasticsearch(["127.0.0.1:9200"])

es.indices.delete(index="wardrive",ignore=404)  #if index exist delete it, or ignore error messages, 404=index not found
docs = df_short.to_dict(orient='records')       #from dataset create a serialize object for import 
bulk(es, docs, index='wardrive',doc_type='wifi', raise_on_error=True) #bulk import
es.indices.refresh() #get import status

{'_shards': {'total': 11, 'successful': 6, 'failed': 0}}